<a href="https://colab.research.google.com/github/sghackathon2022/myrepo/blob/main/ESG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:

!pip install torchvision 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.4 MB/s 
     |████████████████████████████████| 182 kB 70.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.3 MB/s 


In [2]:
from nltk.corpus.reader.tagged import NLTKWordTokenizer
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize,sent_tokenize

df=pd.read_csv('/content/drive/MyDrive/dataset1.csv')
def clean_text(message):
  message=message.lower()
  message=re.sub(r'^https?:\/\/.*[\r\n]*','',message,flags=re.MULTILINE)
  reg_pattern=re.compile(pattern="["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U000026A1"
                                   "]+",flags=re.UNICODE)
  message=reg_pattern.sub(r'',message)

  words=word_tokenize(message)
  stop_words=stopwords.words('english')
  words=[i for i in words if i.strip() !="" and i not in stop_words and i not in string.punctuation]
  #ps=nltk.PorterStemmer()
  wn=nltk.WordNetLemmatizer()
  return " ".join([wn.lemmatize(word) for word in words])
def remove_extraspace(message):
  message=message.lower()
  message=re.sub(r'^https?:\/\/.*[\r\n]*','',message,flags=re.MULTILINE)
  reg_pattern=re.compile(pattern="["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U000026A1"
                                   "]+",flags=re.UNICODE)
  message=reg_pattern.sub(r'',message)

  words=word_tokenize(message)
  return " ".join([i for i in words if i.strip() !="" ])

rm_space=[]
cln_data=[]
for ind in df['News'].index:
  cln_data.append(clean_text(str(df['Title'][ind])+"\n"+str(df['News'][ind])))
  rm_space.append(remove_extraspace(str(df['Title'][ind])+"\n"+str(df['News'][ind])))
df['clean_data']=cln_data
df['data']=rm_space
df.columns
message=df['Title'][27]+"\n"+df["News"][27]
clean_msg=df['clean_data'][27]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
data1="accident happened in Bangalore bommanali, 500 people got injured and 300 were deceased"
data2="I love you"
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline
sent_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)
sent_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
sent_nlp=pipeline("text-classification",model=sent_model,tokenizer=sent_tokenizer)
dict1={"LABEL_0":"NEUTRAL","LABEL_1":"POSITIVE","LABEL_2":"NEGATIVE"}
#data=dict1[sent_nlp(df['clean_data'][0])[0]["label"]]
#data2=dict1[sent_nlp("I hate you")[0]["label"]]
print(sent_nlp(data1)[0],data1)

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

{'label': 'Negative', 'score': 0.7911931276321411} accident happened in Bangalore bommanali, 500 people got injured and 300 were deceased


In [4]:
from transformers import BertForSequenceClassification,AutoTokenizer,pipeline
import torch
id2label= {
    "0": "None",
    "1": "Environmental",
    "2": "Social",
    "3": "Governance"
  }
esg_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-esg", num_labels=4)
esg_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-esg")
esg_nlp=pipeline("text-classification",model=esg_model,tokenizer=esg_tokenizer)
print(esg_nlp(df["clean_data"][0]))

Downloading:   0%|          | 0.00/781 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'Social', 'score': 0.9554513692855835}]


In [3]:
from nltk.corpus.reader.tagged import NLTKWordTokenizer
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer,PorterStemmer,SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize,sent_tokenize
def is_word_exists(sentence,keyword):
  ps = SnowballStemmer("english")
  flag=False
  words=" ".join([ps.stem(str(i).lower()) for i in word_tokenize(sentence)])
  #print(words)
  synonyms=[]
  for syn in wordnet.synsets(keyword):
    for lm in syn.lemmas():
             synonyms.append(lm.name())
  syn = [keyword]+synonyms
  ps_syn=syn+[ps.stem(w) for w in syn] 

  syn=syn+ps_syn
  for word in syn:
    if word is not None and str(word).lower() in words :
      
      flag=True
      break
  return flag
        
def get_environment_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in dict1.keys():
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item1.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if i1_flag:
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if i2_flag:
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature

def get_social_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in ["very serious","serious","medium"]:
      if key =="very serious":
        item3=dict1[key]["direct_object"]
        for i3 in item3.split(","):
          if is_word_exists(sentence,i3.strip()):
            nature=key
            return nature
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item2.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if(i1_flag):
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if (i2_flag):
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature

def get_governance_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in ["very serious","serious","medium"]:
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item2.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if(i1_flag):
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if (i2_flag):
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature  
nature={"environment":{
              "very serious":{"object":"plant, wildlife, habitat, ecosystem","impact":"death,dead, destruction"},
              "serious":{"object":"plant, wildlife, habitat, ecosystem","impact":"debilitation, injury,'injuri', illness, major damage"},
              "medium":{"object":"plant, wildlife, habitat, ecosystem, pollution","impact":"damage, short-term damage"},
              "minimal":{"object":"","impact":""}},
  "social":{
              "very serious":{"object":"human rights, livelihood, life, traditional life, traditional way of life, property","direct_object":"death,crimes, dead,disability, permanent disability,torture, rape, enslavement","impact":"violation, destruction,serious"},
              "serious":{"object":"property,people,mass, livelihood, life, traditional life, traditional way of life, labor rights, labour rights, civil rights, privacy, collective bargaining","impact":"debilitating, damage, impairment, displacement, violation, harm,injuri,injury,illness"},
              "medium":{"object":"injury,'injuri', illness, property, livelihood, life, traditional life, traditional way of life","impact":"treatable, short-term, short term, temporary, non-serious, light, slight"},
              "minimal":{"object":"","impact":""}},
           
  "governance":{
              "very serious":{"object":"national government, economy, bribe, loss, company loss, contracts, contracts value, gains, illegal gains, profit, dishonest profit, corrupt gains, tax, income tax","impact":"destabilise, substantially destabilised, illegal, dishonest, corrupt, avoided, USD 1 billion,  USD 10 billion"},
              "serious":{"object":"company, non-government customer, private customer, government body, public sector, government organisation, pension funds, pension, bribe, contracts","impact":"bankruptcy, USD 100 million, USD 5 billion"},
              "medium":{"object":"company, B2B, property, government fraud","impact":"corruption, fraud, damage, non-serious, slight, light"},
              "minimal": {"object":"","impact":""}}
}
data1="Fire accident happened yesterday night in african forest.1M Animals death and habitat destruction done"
data2="People got fired from infosys organization due to dollar to rupee conversion low in india"
data3="Heavy anti money laundering happened over year in pakistan and goverment taking minimal actions which effects country economy growth"
print(get_environment_nature_impact(nature["environment"],data1))
print(get_social_nature_impact(nature["social"],data2))
print(get_governance_nature_impact(nature["governance"],data3))


very serious
minimal
minimal


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1]:
from nltk.stem import WordNetLemmatizer,PorterStemmer,SnowballStemmer
print(SnowballStemmer("english").stem("injured"))
from nltk.stem import PorterStemmer
e_words= ["wait", "waiting", "waited", "waits","injured","injury","dead","dying"]
ps =SnowballStemmer("english")
for w in e_words:
    rootWord=ps.stem(w)
    print(rootWord)

injur
wait
wait
wait
wait
injur
injuri
dead
die


In [ ]:
from transformers import BertForSequenceClassification,AutoTokenizer
import torch
model = BertForSequenceClassification.from_pretrained(
    '/content/drive/MyDrive/Model', 
    num_labels = 26, #number of classifications
   output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Model')
model.eval()
#text="Donard Trump taken actions against climate change but Upper house rejected his plea. \ still he is pushing withdraw of paris climate change bill"
#text=text.lower()
text=clean_msg
inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            return_tensors="pt"
        )
prediction = model(
            inputs['input_ids'].to(device), 
            token_type_ids=inputs['token_type_ids'].to(device)
        )[0].argmax().item()
id2label= {
    "0": "Bribery & Fraud",
    "1": "Privacy & Data Security",
    "2": "Human Rights & Community-Other",
    "3": "Governance-Other",
    "4": "AntiCompetitive Practices",
    "5": "Governance-Other",
    "6": "Customer Relations",
    "7": "Governance Structures",
    "8": "Discreimination & Workforce Diversity",
    "9": "Health & Safety",
    "10": "Human Rights & Community",
    "11": "Labor Management Relations",
    "12": "Governance-Other",
    "13": "Energy & Climate Change",
    "14": "Product Safety & Quality",
    "15": "Product Safety & Quality",
    "16": "Marketing & Advertising",
    "17": "Supply Chain Management",
    "18": "Governance-Other",
    "19": "Toxic Emissions & Waste/Operational Waste(Non-Hazardous)",
    "20": "Water Stress",
    "21": "Toxic Emissions & Waste",
    "22": "Privacy & Data Security",
    "23": "Biodiversity & Land Use",
    "24": "Energy & Climate Change",
    "25": "Energy & Climate Change"
  }
print(prediction)
print(id2label[str(prediction)])

5
Governance-Other


In [ ]:
#Extract ORG with spacy
import spacy
import re
NER = spacy.load("en_core_web_sm")
#raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
raw_text=df['Title'][27]+"\n"+df["News"][27]
text1= NER(raw_text)
spacy.explain("ORG")
com=[]

for word in text1.ents:
    if word.label_ in ['ORG']:
      com.append(word.text)
com=set(com)
word_count={}
for item in com:
  message=raw_text
  message=message.replace('(','')
  item=item.replace('(','')
  count=len(re.findall('(?='+item+')',message))
  word_count[item]=count
val=0
val_key=""
for key in word_count.keys():
  if val<int(word_count[key]):
    val=int(word_count[key])
    val_key=key

print(val_key)

SBG Management


In [ ]:
!mkdir /content/drive/MyDrive/input_files
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/dataset1.csv')
for ind in range(100):
  open("/content/drive/MyDrive/inpp"+str(ind+1)+".txt",'w').write(str(df['Title'][ind]+"\n"+str(df['News'][ind])))

mkdir: cannot create directory ‘/content/drive/MyDrive/input_files’: File exists


In [4]:
!pip install torchvision 
!pip install transformers
!pip install py_thesaurus
from nltk.corpus.reader.tagged import NLTKWordTokenizer
import pandas as pd
import re
import os
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer,SnowballStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
import re
NER = spacy.load("en_core_web_sm")

from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline

import torch
from nltk.tokenize import word_tokenize,sent_tokenize
def clean_text(message):
  message=message.lower()
  message=re.sub(r'^https?:\/\/.*[\r\n]*','',message,flags=re.MULTILINE)
  reg_pattern=re.compile(pattern="["
                                   u"\U0001F600-\U0001F64F"
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F1E0-\U0001F1FF"
                                   u"\U000026A1"
                                   "]+",flags=re.UNICODE)
  message=reg_pattern.sub(r'',message)

  words=word_tokenize(message)

  stop_words=stopwords.words('english')
  no_stop_words=[i for i in words if i.strip() !="" and i not in stop_words and i not in string.punctuation]
  #ps=nltk.PorterStemmer()
  wn=nltk.WordNetLemmatizer()
  return " ".join([wn.lemmatize(word) for word in no_stop_words])," ".join([i for i in words])


def get_prediction(model_name,message,size,mapping_list):
  prediction="Other"
  try:
    msg_512=message.split(" ")[0:512]
    #print(len(msg_512))
    msg=" ".join([i for i in msg_512])
    
    model = BertForSequenceClassification.from_pretrained(model_name,num_labels = size,output_attentions = False, output_hidden_states = False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer.encode_plus(message,add_special_tokens=True,return_tensors="pt")
    prediction = model(inputs['input_ids'].to(device),token_type_ids=inputs['token_type_ids'].to(device))[0].argmax().item()
  except:
    pass
  return prediction

def get_word_ngrams(message,phrase):
  tokens=message.split(" ")
  tn_phrase=phrase.split(" ")
  values=[]
  if tn_phrase[0] in tokens:
    i=tokens.index(tn_phrase[0])
    j=i+len(tn_phrase)+2
    val=""
    while(i<=j and j<len(tokens)) :
      val=val+" "+tokens[i]
      i=i+1
    i=tokens.index(tn_phrase[0])-2
    j=tokens.index(tn_phrase[0])
    val=""
    while(j>=i and i>=0):
      val=val+" "+tokens[i]
      i=i+1
    val=val+" "+phrase
    values.append(val.strip())
    return list(set(values))
    

    
def get_org_and_key_phrases(raw_text,cln_msg):

  text1= NER(raw_text)
  spacy.explain("ORG")
  com=[]
  key_words=[]
  for word in text1.ents:
    if word.label_ in ['ORG']:
      com.append(word.text)
    elif word.label_ in ['ORDINAL','CORDINAL']:
      key_words.append(word.text)
  com=set(com)
  word_count={}
  for item in com:
    message=raw_text
    message=message.replace('(','')
    if item is not None:
      item=item.replace('(','')
      count=len(re.findall('(?='+item+')',message))
      word_count[item]=count
  val=0
  val_key=""
  for key in word_count.keys():
    if val<int(word_count[key]):
      val=int(word_count[key])
      val_key=key
  phrases=[]
  for w in key_words:
    vals=get_word_ngrams(cln_msg,w)
    if vals is not None and len(vals)>=1:
      phrases=phrases+vals
  return val_key,phrases
def is_word_exists(sentence,keyword):
  ps = SnowballStemmer("english")
  flag=False
  words=" ".join([ps.stem(str(i).lower()) for i in word_tokenize(sentence)])
  #print(words)
  synonyms=[]
  for syn in wordnet.synsets(keyword):
    for lm in syn.lemmas():
             synonyms.append(lm.name())
  syn = [keyword]+synonyms
  ps_syn=syn+[ps.stem(w) for w in syn] 

  syn=syn+ps_syn
  for word in syn:
    if word is not None and str(word).lower() in words :
      
      flag=True
      break
  return flag
        
def get_environment_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in dict1.keys():
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item1.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if i1_flag:
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if i2_flag:
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature

def get_social_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in ["very serious","serious","medium"]:
      if key =="very serious":
        item3=dict1[key]["direct_object"]
        for i3 in item3.split(","):
          if is_word_exists(sentence,i3.strip()):
            nature=key
            return nature
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item2.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if(i1_flag):
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if (i2_flag):
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature

def get_governance_nature_impact(dict1,message):
  
  sentences=sent_tokenize(message)
  nature="minimal"
  ps = PorterStemmer()
  for sentence in sentences:
    for key in ["very serious","serious","medium"]:
      item1=dict1[key]["object"]
      item2=dict1[key]["impact"]
      i1_tokens=item2.split(",")
      i2_tokens=item2.split(",")
      i1_flag=False
      
      for i1 in i1_tokens:
        i1_flag=is_word_exists(sentence,i1.strip())
        if(i1_flag):
          break
      i2_flag=False
      for i2 in i2_tokens:
        i2_flag=is_word_exists(sentence,i2.strip())
        if (i2_flag):
          break
      if i1_flag and i2_flag:
        nature=key
        break

  return nature  
nature_dict={"environment":{
              "very serious":{"object":"plant, wildlife, habitat, ecosystem","impact":"death,dead, destruction"},
              "serious":{"object":"plant, wildlife, habitat, ecosystem","impact":"debilitation, injury,'injuri', illness, major damage"},
              "medium":{"object":"plant, wildlife, habitat, ecosystem, pollution","impact":"damage, short-term damage"},
              "minimal":{"object":"","impact":""}},
  "social":{
              "very serious":{"object":"human rights, livelihood, life, traditional life, traditional way of life, property","direct_object":"death,crimes, dead,disability, permanent disability,torture, rape, enslavement","impact":"violation, destruction,serious"},
              "serious":{"object":"property,people,mass, livelihood, life, traditional life, traditional way of life, labor rights, labour rights, civil rights, privacy, collective bargaining","impact":"debilitating, damage, impairment, displacement, violation, harm,injuri,injury,illness"},
              "medium":{"object":"injury,'injuri', illness, property, livelihood, life, traditional life, traditional way of life","impact":"treatable, short-term, short term, temporary, non-serious, light, slight"},
              "minimal":{"object":"","impact":""}},
           
  "governance":{
              "very serious":{"object":"national government, economy, bribe, loss, company loss, contracts, contracts value, gains, illegal gains, profit, dishonest profit, corrupt gains, tax, income tax","impact":"destabilise, substantially destabilised, illegal, dishonest, corrupt, avoided, USD 1 billion,  USD 10 billion"},
              "serious":{"object":"company, non-government customer, private customer, government body, public sector, government organisation, pension funds, pension, bribe, contracts","impact":"bankruptcy, USD 100 million, USD 5 billion"},
              "medium":{"object":"company, B2B, property, government fraud","impact":"corruption, fraud, damage, non-serious, slight, light"},
              "minimal": {"object":"","impact":""}}
}

##Execution of Code
#df=pd.read_csv('/content/drive/MyDrive/dataset1.csv')
sent_mapping={"LABEL_0":"NEGATIVE","LABEL_1":"POSITIVE"}
esg_mapping={
    "0": "None",
    "1": "Environment",
    "2": "Social",
    "3": "Governance"
  }
sub_pillar_mapping={
    "0": "Bribery & Fraud",
    "1": "Privacy & Data Security",
    "2": "Human Rights & Community-Other",
    "3": "Governance-Other",
    "4": "AntiCompetitive Practices",
    "5": "Governance-Other",
    "6": "Customer Relations",
    "7": "Governance Structures",
    "8": "Discreimination & Workforce Diversity",
    "9": "Health & Safety",
    "10": "Human Rights & Community",
    "11": "Labor Management Relations",
    "12": "Governance-Other",
    "13": "Energy & Climate Change",
    "14": "Product Safety & Quality",
    "15": "Product Safety & Quality",
    "16": "Marketing & Advertising",
    "17": "Supply Chain Management",
    "18": "Governance-Other",
    "19": "Toxic Emissions & Waste/Operational Waste(Non-Hazardous)",
    "20": "Water Stress",
    "21": "Toxic Emissions & Waste",
    "22": "Privacy & Data Security",
    "23": "Biodiversity & Land Use",
    "24": "Energy & Climate Change",
    "25": "Energy & Climate Change"
  }
label2id= {
    "Access_And_Affordability": 2,
    "Air_Quality": 21,
    "Business_Ethics": 0,
    "Business_Model_Resilience": 3,
    "Competitive_Behavior": 4,
    "Critical_Incident_Risk_Management": 5,
    "Customer_Privacy": 22,
    "Customer_Welfare": 6,
    "Data_Security": 1,
    "Director_Removal": 7,
    "Ecological_Impacts": 23,
    "Employee_Engagement_Inclusion_And_Diversity": 8,
    "Employee_Health_And_Safety": 9,
    "Energy_Management": 24,
    "GHG_Emissions": 25,
    "Human_Rights_And_Community_Relations": 10,
    "Labor_Practices": 11,
    "Management_Of_Legal_And_Regulatory_Framework": 12,
    "Physical_Impacts_Of_Climate_Change": 13,
    "Product_Design_And_Lifecycle_Management": 15,
    "Product_Quality_And_Safety": 14,
    "Selling_Practices_And_Product_Labeling": 16,
    "Supply_Chain_Management": 17,
    "Systemic_Risk_Management": 18,
    "Waste_And_Hazardous_Materials_Management": 19,
    "Water_And_Wastewater_Management": 20
  }
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline
sent_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)
sent_tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")

esg_model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg', num_labels=4)
esg_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
esg_nlp=pipeline("text-classification",model=esg_model,tokenizer=esg_tokenizer)

pillar_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Model', num_labels=26)
pillar_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Model")
pillar_nlp=pipeline("text-classification",model=pillar_model,tokenizer=pillar_tokenizer)
#Added Extra two columns have data with stop words and with out stop words
esg_data=[]
pillar_data=[]
org_data=[]
key_phrases=[]
esg_model='yiyanghkust/finbert-esg'
pillar_model='/content/drive/MyDrive/Model'
files=[]
data=[]
controversy=[]
nature=[]
for file in os.listdir('/content/drive/MyDrive/input_files'):
    # check if current path is a file
    if os.path.isfile(os.path.join('/content/drive/MyDrive/input_files', file)):
        files.append(os.path.join('/content/drive/MyDrive/input_files', file))
for file in files:
  print(file)
  msg=open(file,"r").read()
  data.append(msg)
  msg=msg.replace("(","").replace(")","")
  cln_msg,full_msg=clean_text(msg)
  try:
    controversy.append(sent_nlp(cln_msg)["label"])
  except:
    controversy.append("NEUTRAL")
  try:

    val=esg_mapping[esg_nlp(cln_msg)[0]["label"]].lower()
    if val == "environment":
      nature.append(get_environment_nature_impact(nature_dict[val],msg))
    elif val == "social":
      nature.append(get_social_nature_impact(nature_dict[val],msg))
    elif val == "governance":
      nature.append(get_governance_nature_impact(nature_dict[val],msg))
    esg_data.append(val)
  except:
     esg_data.append("None")
     nature.append(" ")
  try:
    pillar1=pillar_nlp(cln_msg)[0]["label"]  
    print(pillar1)
    pillar_data.append(sub_pillar_mapping[label2id[pillar1]])
  except:
     pillar_data.append("None") 
  org,phrases=get_org_and_key_phrases(full_msg,cln_msg)
  org_data.append(org)
  key_phrases.append(phrases)
df=pd.DataFrame(data,columns=["data"])
df['controversy']=controversy
df['key_phrases']=key_phrases 
df['esg_type']=esg_data
df['sub_pillar']=pillar_data
df['org']=org_data
df['nature']=nature
df2=df[['data','controversy','esg_type','nature','sub_pillar','org','key_phrases']]
json=df2.to_json()
df2.to_csv('output.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.9 MB/s 
     |████████████████████████████████| 182 kB 53.8 MB/s 
     |████████████████████████████████| 7.6 MB 29.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for py-thesaurus: filename=py_thesaurus-1.0.5-py3-none-any.whl size=6275 sha256=b606a8aa64c46dcd722a21aa70f7460bc6d0dac6037ab3d1205443374a29a3f3
  Stored in directory: /root/.cache/pip/wheels/d8/a6/ca/a21cfc89709d4beb103360352738166829485d882c41187912
Successfully built py-thesaurus


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

/content/drive/MyDrive/input_files/1.txt
Labor_Practices
/content/drive/MyDrive/input_files/2.txt
Management_Of_Legal_And_Regulatory_Framework
/content/drive/MyDrive/input_files/3.txt
Business_Ethics
/content/drive/MyDrive/input_files/4.txt


Token indices sequence length is longer than the specified maximum sequence length for this model (812 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (812 > 512). Running this sequence through the model will result in indexing errors


Employee_Engagement_Inclusion_And_Diversity
/content/drive/MyDrive/input_files/5.txt
/content/drive/MyDrive/input_files/6.txt
Competitive_Behavior
/content/drive/MyDrive/input_files/7.txt
/content/drive/MyDrive/input_files/8.txt
Physical_Impacts_Of_Climate_Change
/content/drive/MyDrive/input_files/9.txt
Energy_Management
/content/drive/MyDrive/input_files/10.txt
Employee_Health_And_Safety
/content/drive/MyDrive/input_files/11.txt
Employee_Health_And_Safety
/content/drive/MyDrive/input_files/12.txt
Business_Ethics
/content/drive/MyDrive/input_files/13.txt
/content/drive/MyDrive/input_files/14.txt
Business_Ethics
/content/drive/MyDrive/input_files/15.txt
Employee_Health_And_Safety
/content/drive/MyDrive/input_files/16.txt
Employee_Health_And_Safety
/content/drive/MyDrive/input_files/17.txt
Management_Of_Legal_And_Regulatory_Framework
/content/drive/MyDrive/input_files/18.txt
Business_Ethics
/content/drive/MyDrive/input_files/19.txt
Employee_Health_And_Safety
/content/drive/MyDrive/input_



```
# This is formatted as code
```

